# Twitter US Airline Sentiment Classification
Using Word2Vec and Multiclass Logistic Regression to predict sentiment from tweets.

In [7]:
# Step 1: Upload the file manually in Colab
from google.colab import files
uploaded = files.upload()


Saving Tweets.csv to Tweets.csv


In [3]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

In [2]:
%pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 10.8 MB/s eta 0:00:00


In [10]:
import pandas as pd
import numpy as np
import re
import string
import gensim
import nltk
import gensim.downloader as api
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import contractions

# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Load Dataset

In [8]:
# Load dataset (replace with actual path if needed)
df = pd.read_csv("Tweets.csv")[['airline_sentiment', 'text']]
df.columns = ['sentiment', 'tweet']
df.head()

,sentiment,tweet
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


## Preprocess Tweets

In [11]:
def clean_tweet(text):
    text = contractions.fix(text)  # Expand contractions
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"@\w+|#\w+", '', text)
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

df['tokens'] = df['tweet'].apply(clean_tweet)
df[['tweet', 'tokens']].head()

,tweet,tokens
0,@VirginAmerica What @dhepburn said.,[said]
1,@VirginAmerica plus you've added commercials t...,"[plus, added, commercial, experience, tacky]"
2,@VirginAmerica I didn't today... Must mean I n...,"[today, must, mean, need, take, another, trip]"
3,@VirginAmerica it's really aggressive to blast...,"[really, aggressive, blast, obnoxious, enterta..."
4,@VirginAmerica and it's a really big bad thing...,"[really, big, bad, thing]"


## Load Google News Word2Vec Model

In [12]:
# This downloads ~1.6GB model the first time
w2v_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


## Vectorize Tweets

In [13]:
def vectorize_tokens(token_lists, w2v_model, vector_size=300):
    vectors = []
    for tokens in token_lists:
        word_vectors = [w2v_model[word] for word in tokens if word in w2v_model]
        if word_vectors:
            vectors.append(np.mean(word_vectors, axis=0))
        else:
            vectors.append(np.zeros(vector_size))
    return np.array(vectors)

X = vectorize_tokens(df['tokens'], w2v_model)
y = df['sentiment']

## Train-Test Split and Model Training

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(f"Accuracy on test set: {accuracy_score(y_test, y_pred):.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy on test set: 0.7725


## Predict Tweet Sentiment Function

In [17]:
def predict_tweet_sentiment(model, w2v_model, tweet):
    tokens = clean_tweet(tweet)
    word_vectors = [w2v_model[word] for word in tokens if word in w2v_model]
    if word_vectors:
        vector = np.mean(word_vectors, axis=0).reshape(1, -1)
    else:
        vector = np.zeros((1, 300))
    prediction = model.predict(vector)
    return prediction[0]

# Test example
predict_tweet_sentiment(clf, w2v_model, "JetBlue is the best airline I've ever flown with!")

'positive'